In [0]:
# TODO
# change picture to hive database
# create features
# 1. time to next appointment
# 2. split date and time into 2 columns
# 3. create treatment col (there seems to be different treatment processes)
# create "patient / date - gantt in power bi"

# JOB Interview - Data Analyst

Welcome to this notebook! Here I present the approach I took to analyse the dataset the interviewers sent to me during a "Data Analyst" job interview process. The R&D department operates in the healthcare sector and produces hearing aids for B2B. The following steps were taken:
<br><br>
1. Understanding the Problem & Setting the Scene
2. Setting up the infrastructure
3. Getting Data
4. Exploratory Data Analysis - Python
    1. Sneak Peak
    2. Types of columns & description
    3. Descriptive Statistics
    4. Plots & Questions
    5. Outlier Detection
5. Feature Engineering
6. Interpretation
7. Future Work & Limitations
8. Sources Used

## 1. Understanding the Problem

The interviewers have provided a dataset in the ```.csv```-format to the potential candidate (me). The formulated task, by the interviewers, is to "showcase your skills and investigate the dataset". The interview is scheduled to take place a week after receiving the dataset. In this appointment the interviewee will need to present his results. Emphasis is placed on written code. <b>No further instructions have been made. </b> The R&D department offering the Data Analyst position are looking for support due to the growing amount of requests.

However, the interviewee has deducted that the following requirements need to be met to some extent, in order for him to proceed to the interview stage 2. He wants to use this interview as an opportunity to learn new things. He bases his assumptions on information provided in a screening interview and the job position:

  * a) Company language is English, therefore code and explanations should meet this requirement
  * b) Programming skills in Python, SQL as well Power BI (DAX / M-Language) should be incorporated, as the department relies on these for coding
  * c) Coding principles should be followed ("Commit early, commit often", commented code, clean code)
  * d) Deliver high quality analytics in easy and understandable way A
  * e) Answering common data analytics questions using EDA for the dataset
  * f) Data pipelines should be implemented
  * g) Utilizing Apache Spark (pyspark), Azure Services and ML methods are icing on the cake!

## 2. Setting up the infrastructure

[![IMAGE ALT TEXT HERE](https://github.com/sibr1011/job_interview/blob/main/Slide1.png?raw=true)](https://github.com/sibr1011/job_interview/blob/main/Slide1.png?raw=true)

The image above illustrates the process that was taken for the task at hand. The provided ```.csv``` was uploaded as Hive Database. This section focues on Problems f) and partly g) from section *1. Understanding the Problem.*

## 3. Getting Data

In [0]:
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row
appName= "hive_metastore"
master= "local"

In [0]:
# initialize session 
spark = SparkSession.builder \
	.master(master).appName(appName).getOrCreate()

In [0]:
# load data using spark sql as pyspark.sql. df and pandas.dF
df_old = spark.sql("select patient, ` action` as action, ` org:resource` as org_resource, ` DateTime` as date_time from wsa_jobinterviewdata")
pandasDF = df.toPandas()

In [0]:
# i noticed schema was changed through upload, so i changed the col names
# df_old.write.saveAsTable('wsa_jobinterview')
df = spark.sql("select * from wsa_jobinterview")

In [0]:
# now the schema is the way we want it
print(df.printSchema())

root
 |-- patient: string (nullable = true)
 |-- action: string (nullable = true)
 |-- org_resource: string (nullable = true)
 |-- date_time: timestamp (nullable = true)

None


In [0]:
type(df)

Out[131]: pyspark.sql.dataframe.DataFrame

## 4. Exploratory Data Analysis

### 4.1 Sneak Peak

In [0]:
# 1. shape -> 690 rows, 4 columns
print("records: ", df.count())
print("columns: ", sum(1 for x in df.columns))

records:  690
columns:  4


In [0]:
# 1. head() - i think the output looks better in pandas df
pandasDF.head(10)

,patient,action,org_resource,date_time
0,patient 0,First consult,Dr. Anna,2017-01-02 11:40:11
1,patient 0,Blood test,Lab,2017-01-02 12:47:33
2,patient 0,Physical test,Nurse Jesse,2017-01-02 12:53:50
3,patient 0,Second consult,Dr. Anna,2017-01-02 16:21:06
4,patient 0,Surgery,Dr. Charlie,2017-01-05 13:23:09
5,patient 0,Final consult,Dr. Ben,2017-01-09 08:29:28
6,patient 1,First consult,Dr. Anna,2017-01-02 12:50:35
7,patient 1,Physical test,Nurse Jesse,2017-01-02 13:59:14
8,patient 1,Blood test,Lab,2017-01-02 14:20:19
9,patient 1,X-ray scan,Team 1,2017-01-06 09:13:40


In [0]:
# 1. tail - same here
pandasDF.tail(10)

,patient,action,org_resource,date_time
680,patient 98,Second consult,Dr. Anna,2017-05-26 11:53:12
681,patient 98,Surgery,Dr. Alex,2017-05-30 14:13:17
682,patient 98,Final consult,Dr. Ben,2017-05-31 14:16:13
683,patient 99,First consult,Dr. Bob,2017-05-18 11:57:11
684,patient 99,Blood test,Lab,2017-05-18 13:13:44
685,patient 99,X-ray scan,Team 2,2017-05-25 11:03:27
686,patient 99,Physical test,Nurse Corey,2017-05-25 11:55:24
687,patient 99,Second consult,Dr. Anna,2017-05-29 15:12:39
688,patient 99,Medicine,Pharmacy,2017-05-29 16:17:13
689,patient 99,Final consult,Dr. Anna,2017-05-31 12:46:43


### 4.2 Types of columns and description

In [0]:
# schema - 4 vars.
df.columns

Out[135]: ['patient', 'action', 'org_resource', 'date_time']

In [0]:
# get descriptive statistics for all columns
print(df.select('patient').describe().show())
print(df.select('action').describe().show())
print(df.select('org_resource').describe().show())
print(df.select('date_time').describe().show())

+-------+----------+
|summary|   patient|
+-------+----------+
|  count|       690|
|   mean|      null|
| stddev|      null|
|    min| patient 0|
|    max|patient 99|
+-------+----------+

None
+-------+-----------+
|summary|     action|
+-------+-----------+
|  count|        690|
|   mean|       null|
| stddev|       null|
|    min| Blood test|
|    max| X-ray scan|
+-------+-----------+

None
+-------+------------+
|summary|org_resource|
+-------+------------+
|  count|         690|
|   mean|        null|
| stddev|        null|
|    min|    Dr. Alex|
|    max|      Team 2|
+-------+------------+

None
+-------+
|summary|
+-------+
|  count|
|   mean|
| stddev|
|    min|
|    max|
+-------+

None


In [0]:
# any missing values?
print(pandasDF.isnull().values.any())

False
